# **Algoritmo apriori**

En esta libreta exploraremos el modelo mercado canasta basado en el algoritmo apriori usando una base de datos de transacciones de una tienda de productos comestibles.


In [ ]:
import pandas as pd

## Carga de datos
Primero leemos la base de datos [Online Retail II](https://archive.ics.uci.edu/ml/datasets/Online+Retail+II) de [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php) y mostramos algunos registros.

In [ ]:
tdb_nan = pd.read_excel('https://archive.ics.uci.edu/ml/machine-learning-databases/00502/online_retail_II.xlsx')
tdb_nan.head(5)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


Eliminamos los registros inválidos y revisamos el número de registros.



In [ ]:
print(tdb_nan.shape)
tdb = tdb_nan.dropna(axis=0)
print(tdb.shape)

(525461, 8)
(417534, 8)


La base de datos cuenta con un registro por cada producto comprado (campo `Description`) y una factura asociada (`Invoice`), ademas de otros campos. Para nuestro ejercicio, vamos a utilizar únicamente `Invoice` y `Description`.

In [ ]:
tdb = tdb[['Invoice', 'Description']]
tdb.head(5)

,Invoice,Description
0,489434,15CM CHRISTMAS GLASS BALL 20 LIGHTS
1,489434,PINK CHERRY LIGHTS
2,489434,WHITE CHERRY LIGHTS
3,489434,"RECORD FRAME 7"" SINGLE SIZE"
4,489434,STRAWBERRY CERAMIC TRINKET BOX


Para encontrar elementos frecuentes, es necesario agrupar los productos comprados en una misma transacción usando el número de factura `Invoice`.

In [ ]:
tdb = tdb.groupby(['Invoice'])['Description'].apply(set).reset_index(name='Description')
tdb.head(5)

,Invoice,Description
0,489434,"{PINK DOUGHNUT TRINKET POT , 15CM CHRISTMAS GL..."
1,489435,"{HEART MEASURING SPOONS LARGE, LUNCHBOX WITH C..."
2,489436,"{ PEACE WOODEN BLOCK LETTERS, FULL ENGLISH BRE..."
3,489437,"{RETRO COFFEE MUGS ASSORTED, WOODEN BOX ADVENT..."
4,489438,"{JUMBO BAG CHARLIE AND LOLA TOYS, JUMBO BAG TO..."


Revisamos el número de transacciones totales en la base de datos.

In [ ]:
d = len(set(e for t in tdb['Description'] for e in t))
n = len(tdb)

print('Número de elementos únicos: {0}\nNúmero de transacciones: {1}'.format(d, n))

Número de elementos únicos: 4459
Número de transacciones: 23587


## Ejercicio
Programa el algoritmo Apriori y realiza la búsqueda de conjuntos con cardinalidad 1, 2 y 3 de elementos frecuentes usando diferentes soportes mínimos para las transacciones de la base de datos Online Retail II.

In [ ]:
X = tdb.iloc[:200,:]
X

,Invoice,Description
0,489434,"{PINK DOUGHNUT TRINKET POT , 15CM CHRISTMAS GL..."
1,489435,"{HEART MEASURING SPOONS LARGE, LUNCHBOX WITH C..."
2,489436,"{ PEACE WOODEN BLOCK LETTERS, FULL ENGLISH BRE..."
3,489437,"{RETRO COFFEE MUGS ASSORTED, WOODEN BOX ADVENT..."
4,489438,"{JUMBO BAG CHARLIE AND LOLA TOYS, JUMBO BAG TO..."
...,...,...
195,489876,"{RETRO BLUE SPOTTY WASHING UP GLOVES, 12 MINI ..."
196,489877,"{ WHITE CHERRY LIGHTS, HEART IVORY TRELLIS SMA..."
197,489878,"{VINTAGE BEAD NOTEBOOK, BEWARE OF THE CAT META..."
198,489879,"{SILVER SLEIGHBELL WREATH, ENGLISH ROSE DESIG..."


In [ ]:
def algoritmo_apriori(X,soporte_=0.5):
    #Definimos el soporte en y las variables auxiliares
    no_transacciones = X.shape[0]
    soporte = int(soporte_*no_transacciones//1 + 1)

    #PASOS PARA C1 Y F1

    #Creamos las variables y las variables auxiliares
    C1 = {}
    F1 = []

    #Guardamos los productos en un diccionario donde la llave son los productos y los valores son el número de transacciones en las que aparece
    for j in range(no_transacciones):
        for j1 in X.iloc[j,1]:
            if j1 in C1:
    
                C1[str(j1)] += 1

                #Cuando pasan el soporte se añaden a F1
                if C1[str(j1)] > soporte and not (str(j1) in F1):
                    F1.append(str(j1)) 
            else:
                C1[str(j1)] = 1

    print('El conjunto F1 es ',F1)    
    print('El tamaño del conjunto F1 es ',len(F1))

    #PASOS PARA C2 Y F2

    #Creamos las variables y las variables auxiliares
    C2 = {}
    F2_ = {}
    F2 = []

    #Se toman los valores de F1 y posteriormente se meten todas las combinaciones sin repetir a C2
    for k in range(len(F1)-1):
        for k1 in range(k,len(F1)):
            j = F1[k]
            j1 = F1[k1]
            if j != j1:
                if not ((j,j1) in C2) and not ((j1,j) in C2):
                    C2[(j,j1)] = 0

    #Se usa como variable auxiliar F2_ para contar el número de veces que aparecen cada tupla en la misma transacción
    for j,v in C2.keys():
        for j1 in range(no_transacciones):
            a_ = X.iloc[j1,1]
            if j in a_ and v in a_:
                if (j,v) in F2_:
                    F2_[(j,v)] += 1
                else:
                    F2_[(j,v)] = 1

    #Con el cuenta hecha en F2_ se guardan los valores en F2 que pasaron el umbral
    for j,v in F2_.items():
        if v > soporte:
            F2.append(j)


    print('El conjunto F2 es ',F2)
    print('El tamaño del conjunto F2 es ',len(F2))

    #PASOS PARA V3 Y F3

    #Creamos las variables y las variables auxiliares
    C3 = {}
    F3_ = {}
    F3 = []

    #Se toman los pares que estan en F2 y se proceden a añadir tripletas con el uso de estos valores tomando en cuenta la cerradura y se añaden a C3
    for k1 in range(len(F2)-1):
        for k2  in range(k1+1,len(F2)):
            j1 = F2[k1]
            j2 = F2[k2]

            #Se revisa si dos tuplas comparten un elemento en común y además si son distintas, si los valores que no son iguales se
            #aparece como par en F2 entonces podemos hacer la unirlos manteniendo la cerradura
            if (j1[0] in j2 or j1[1] in j2) and j1 != j2:


                #Casos como pueden venir acomodados estos valores
                if j1[0] == j2[0]:
                    if (j1[1],j2[1]) in F2 or (j2[1],j1[1]) in F2:
                        A = [j1[0],j1[1],j2[1]]
                        A = tuple(sorted(A))
                        C3[A] = 0

                elif j1[0] == j2[1]:
                    if (j1[1],j2[0]) in F2 or (j2[0],j1[1]) in F2:
                        A = [j1[0],j1[1],j2[0]]
                        A = tuple(sorted(A))
                        C3[A] = 0
                            

                elif j1[1] == j2[0]:
                    if (j1[0],j2[1]) in F2 or (j2[1],j1[0]) in F2:
                        A = [j1[0],j1[1],j2[1]]
                        A = tuple(sorted(A))
                        C3[A] = 0

                elif j1[1] == j2[1]:
                    if (j1[0],j2[0]) in F2 or (j2[0],j1[0]) in F2:
                        A = [j1[0],j1[1],j2[0]]
                        A = tuple(sorted(A))
                        C3[A] = 0

    #Contamos el número de veces que aparecen las tripletas en la misma transacción y guardamos la información con ayuda de F3_
    for j,v,k in C3.keys():
        for j1 in range(no_transacciones):
            a_ = X.iloc[j1,1]
            if j in a_ and v in a_ and k in a_:
                b_ = tuple(sorted([j,v,k]))
                if b_ in F2_:
                    F3_[b_] += 1
                else:
                    F3_[b_] = 1

    #Guardamos en F3 todas las tripletas que pasaron el umbral
    for j,v in F3_.items():
        if v > soporte:
            F3.append(j)

    print('El conjunto F3 es ',F3)
    print('El tamaño del conjunto F3 es ',len(F3))

In [ ]:
algoritmo_apriori(X,0.04)

El conjunto F1 es  ["PAPER CHAIN KIT 50'S CHRISTMAS ", 'SCOTTIE DOG HOT WATER BOTTLE', 'ANTIQUE SILVER TEA GLASS ETCHED', 'BAKING SET 9 PIECE RETROSPOT ', 'WHITE HANGING HEART T-LIGHT HOLDER', 'GREY HEART HOT WATER BOTTLE', 'RETRO SPOT TEA SET CERAMIC 11 PC ', 'UNION JACK HOT WATER BOTTLE ', 'STRAWBERRY CERAMIC TRINKET BOX', 'JUMBO BAG RED WHITE SPOTTY ', 'HOT WATER BOTTLE TEA AND SYMPATHY', 'RED SPOTTY COIR DOORMAT', 'ASSORTED COLOUR BIRD ORNAMENT', ' WHITE CHERRY LIGHTS', 'PACK OF 72 RETRO SPOT CAKE CASES', 'PLEASE ONE PERSON  METAL SIGN', 'RED SPOTTY CUP', 'SWEETHEART CERAMIC TRINKET BOX', 'RED SPOT HEART HOT WATER BOTTLE', 'PAPER CHAIN KIT RETRO SPOT', "SET OF THREE 50'S GIFT WRAPS ", 'RED/WHITE DOTS RUFFLED UMBRELLA', 'REX CASH+CARRY JUMBO SHOPPER', 'RETRO SPOT TRADITIONAL TEAPOT ', 'JUMBO STORAGE BAG SUKI', 'WHITE SKULL HOT WATER BOTTLE ', 'CHOCOLATE HOT WATER BOTTLE', 'FAIRY CAKE CANDLES', 'SET OF THREE VINTAGE GIFT WRAPS', ' VINTAGE DESIGN GIFT TAGS', 'HOME BUILDING BLOCK WORD'

In [ ]:
algoritmo_apriori(X,0.01)

El conjunto F1 es  ['SCOTTIE DOG HOT WATER BOTTLE', 'BAKING SET 9 PIECE RETROSPOT ', 'UNION JACK HOT WATER BOTTLE ', 'RETRO RED SPOTTY WASHING UP GLOVES', 'WHITE HANGING HEART T-LIGHT HOLDER', 'PARTY CONE CHRISTMAS DECORATION ', "PAPER CHAIN KIT 50'S CHRISTMAS ", 'PLEASE ONE PERSON  METAL SIGN', ' WHITE CHERRY LIGHTS', 'HOT WATER BOTTLE TEA AND SYMPATHY', 'ASSORTED COLOUR BIRD ORNAMENT', 'POSTAGE', 'VINTAGE SNAKES & LADDERS', 'SET OF THREE VINTAGE GIFT WRAPS', 'PAPER CHAIN KIT RETRO SPOT', 'ANTIQUE SILVER TEA GLASS ETCHED', 'COFFEE MUG DOG + BALL DESIGN', 'RETRO SPOT TEA SET CERAMIC 11 PC ', 'PINK BLUE FELT CRAFT TRINKET BOX', 'STRAWBERRY CERAMIC TRINKET BOX', 'VINTAGE HEADS AND TAILS CARD GAME ', 'RED HANGING HEART T-LIGHT HOLDER', 'CHARLIE+LOLA PINK HOT WATER BOTTLE', 'HOME BUILDING BLOCK WORD', 'DOOR MAT TOPIARY', 'RED SPOT HEART HOT WATER BOTTLE', 'SWEETHEART CERAMIC TRINKET BOX', ' VINTAGE DESIGN GIFT TAGS', 'JUMBO BAG RED WHITE SPOTTY ', 'KINGS CHOICE CIGAR BOX MATCHES ', 'FANCY 

In [ ]:
algoritmo_apriori(X,0.005)

El conjunto F1 es  ['SCOTTIE DOG HOT WATER BOTTLE', 'BAKING SET 9 PIECE RETROSPOT ', 'PARTY CONE CHRISTMAS DECORATION ', 'POSTAGE', 'RETRO RED SPOTTY WASHING UP GLOVES', 'UNION JACK HOT WATER BOTTLE ', 'WHITE HANGING HEART T-LIGHT HOLDER', "PAPER CHAIN KIT 50'S CHRISTMAS ", 'RETRO SPOT TEA SET CERAMIC 11 PC ', 'JUMBO BAG PINK VINTAGE PAISLEY', 'CHARLIE+LOLA PINK HOT WATER BOTTLE', 'PINK BLUE FELT CRAFT TRINKET BOX', 'PLEASE ONE PERSON  METAL SIGN', 'VINTAGE SNAKES & LADDERS', ' WHITE CHERRY LIGHTS', 'SET OF THREE VINTAGE GIFT WRAPS', 'PAPER CHAIN KIT RETRO SPOT', 'HOT WATER BOTTLE TEA AND SYMPATHY', 'PAINTED BIRD ASSORTED CHRISTMAS', 'AREA PATROLLED METAL SIGN', 'ASSORTED COLOUR BIRD ORNAMENT', 'LADIES & GENTLEMEN METAL SIGN', 'ANTIQUE SILVER TEA GLASS ETCHED', 'RED TOADSTOOL LED NIGHT LIGHT', 'BIRD DECORATION RED SPOT', 'RED HANGING HEART T-LIGHT HOLDER', 'RETRO SPOT LAMP', 'COFFEE MUG DOG + BALL DESIGN', 'ANTIQUE LILY FAIRY LIGHTS', 'KINGS CHOICE SMALL TUBE MATCHES', 'CHOCOLATE HOT W